In [1]:
!pip install ipython-sql prettytable

import prettytable

prettytable.DEFAULT = 'DEFAULT'

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----------------------------------- ---- 1.8/2.1 MB 8.9 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 7.4 MB/s eta 0:00:00
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
import sqlite3

# Establish a connection to the SQLite database
con = sqlite3.connect("FinalDB.db")

In [9]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [11]:
# Load datasets into DataFrames
census_data = pd.read_csv("ChicagoCensusData.csv")
public_schools_data = pd.read_csv("ChicagoPublicSchools.csv")
crime_data = pd.read_csv("ChicagoCrimeData.csv")

# Save DataFrames to database tables
census_data.to_sql("CENSUS_DATA", con, if_exists="replace", index=False)
public_schools_data.to_sql("CHICAGO_PUBLIC_SCHOOLS", con, if_exists="replace", index=False)
crime_data.to_sql("CHICAGO_CRIME_DATA", con, if_exists="replace", index=False)

print("Data successfully stored in the database!")


Data successfully stored in the database!


In [12]:
%sql sqlite:///FinalDB.db

In [13]:
# Problem 1: Find the total number of crimes recorded in the CRIME table
query = "SELECT COUNT(*) AS total_crimes FROM CHICAGO_CRIME_DATA;"
result = pd.read_sql_query(query, con)
print("Total number of crimes recorded:")
print(result)

Total number of crimes recorded:
   total_crimes
0           533


In [14]:
# Problem 2
# List community area names and numbers with per capita income less than 11000.
query2="SELECT COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER FROM CENSUS_DATA WHERE PER_CAPITA_INCOME<11000; "
result2=pd.read_sql_query(query2,con)
print(result2)

  COMMUNITY_AREA_NAME  COMMUNITY_AREA_NUMBER
0  West Garfield Park                   26.0
1      South Lawndale                   30.0
2         Fuller Park                   37.0
3           Riverdale                   54.0


In [18]:
query_minors = """
SELECT Case_Number 
FROM CHICAGO_CRIME_DATA 
WHERE Description LIKE '%minor%';
"""
result_minors = pd.read_sql_query(query_minors, con)
print("Case numbers for crimes involving minors:")
print(result_minors)

Case numbers for crimes involving minors:
  CASE_NUMBER
0    HL266884
1    HK238408


In [19]:
# Problem 4: List all kidnapping crimes involving a child
query_kidnapping_child = """
SELECT Case_Number, Primary_Type, Description 
FROM CHICAGO_CRIME_DATA 
WHERE Primary_Type = 'KIDNAPPING' AND Description LIKE '%child%';
"""
result_kidnapping_child = pd.read_sql_query(query_kidnapping_child, con)
print("Kidnapping crimes involving a child:")
print(result_kidnapping_child)


Kidnapping crimes involving a child:
  CASE_NUMBER PRIMARY_TYPE               DESCRIPTION
0    HN144152   KIDNAPPING  CHILD ABDUCTION/STRANGER


In [27]:
query_crimes_at_schools = """
SELECT DISTINCT Primary_Type 
FROM CHICAGO_CRIME_DATA 
WHERE Location_Description LIKE '%SCHOOL%';
"""
result_crimes_at_schools = pd.read_sql_query(query_crimes_at_schools, con)
print("Kinds of crimes recorded at schools:")
print(result_crimes_at_schools)


Kinds of crimes recorded at schools:
             PRIMARY_TYPE
0                 BATTERY
1         CRIMINAL DAMAGE
2               NARCOTICS
3                 ASSAULT
4       CRIMINAL TRESPASS
5  PUBLIC PEACE VIOLATION


In [28]:
# Problem 6: List the type of schools along with the average safety score for each type
query_avg_safety_score = """
SELECT "Elementary, Middle, or High School" AS School_Type, AVG(Safety_Score) AS Average_Safety_Score
FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY "Elementary, Middle, or High School";
"""
result_avg_safety_score = pd.read_sql_query(query_avg_safety_score, con)
print("Average safety score for each type of school:")
print(result_avg_safety_score)

Average safety score for each type of school:
  School_Type  Average_Safety_Score
0          ES             49.520384
1          HS             49.623529
2          MS             48.000000


In [29]:
# Problem 7: List 5 community areas with the highest percentage of households below poverty line
query_highest_poverty = """
SELECT Community_Area_Name, Percent_Households_Below_Poverty 
FROM CENSUS_DATA 
ORDER BY Percent_Households_Below_Poverty DESC 
LIMIT 5;
"""
result_highest_poverty = pd.read_sql_query(query_highest_poverty, con)
print("Top 5 community areas with highest % of households below poverty line:")
print(result_highest_poverty)

Top 5 community areas with highest % of households below poverty line:
  COMMUNITY_AREA_NAME  PERCENT_HOUSEHOLDS_BELOW_POVERTY
0           Riverdale                              56.5
1         Fuller Park                              51.2
2           Englewood                              46.6
3      North Lawndale                              43.1
4  East Garfield Park                              42.4


In [30]:
# Problem 8: Which community area is most crime prone? Display the community area number only
query_most_crime_prone = """
SELECT Community_Area_Number 
FROM CHICAGO_CRIME_DATA 
GROUP BY Community_Area_Number 
ORDER BY COUNT(*) DESC 
LIMIT 1;
"""
result_most_crime_prone = pd.read_sql_query(query_most_crime_prone, con)
print("Community area number most crime prone:")
print(result_most_crime_prone)


Community area number most crime prone:
   COMMUNITY_AREA_NUMBER
0                   25.0


In [31]:
# Problem 9: Find the name of the community area with highest hardship index using a sub-query
query_highest_hardship = """
SELECT Community_Area_Name 
FROM CENSUS_DATA 
WHERE Hardship_Index = (SELECT MAX(Hardship_Index) FROM CENSUS_DATA);
"""
result_highest_hardship = pd.read_sql_query(query_highest_hardship, con)
print("Community area with highest hardship index:")
print(result_highest_hardship)


Community area with highest hardship index:
  COMMUNITY_AREA_NAME
0           Riverdale


In [32]:
# Problem 10: Find the community area name with most number of crimes using a sub-query
query_most_crime_area_name = """
SELECT Community_Area_Name 
FROM CENSUS_DATA 
WHERE Community_Area_Number = (SELECT Community_Area_Number 
                                FROM CHICAGO_CRIME_DATA 
                                GROUP BY Community_Area_Number 
                                ORDER BY COUNT(*) DESC 
                                LIMIT 1);
"""
result_most_crime_area_name = pd.read_sql_query(query_most_crime_area_name, con)
print("Community area with the most number of crimes:")
print(result_most_crime_area_name)

Community area with the most number of crimes:
  COMMUNITY_AREA_NAME
0              Austin
